# 5. 머신러닝 프로세스2: 데이터셋 분할과 모델검증
- 훈련데이터 및 테스트 데이터셋 분할
- 홀드아웃과 교차검증

## 5-1. 특성치(X), 레이블(y) 나누기

In [2]:
# 데이터셋 불러오기
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
data = pd.read_csv("../제공자료/Fvote.csv", encoding='utf-8')
data.head()

,gender_female,gender_male,region_Chungcheung,region_Honam,region_Others,region_Sudo,region_Youngnam,edu,income,age,score_gov,score_progress,score_intention,vote,parties
0,0,1,0,0,0,0,1,3,3,3,2,2,4.0,1,2
1,0,1,0,0,1,0,0,2,3,3,2,4,3.0,0,3
2,0,1,0,1,0,0,0,1,2,4,1,3,2.8,1,4
3,1,0,0,0,0,1,0,2,1,3,5,4,2.6,1,1
4,0,1,0,0,0,1,0,1,2,4,4,3,2.4,1,1


In [5]:
# 특성변수 데이터셋 나누기
# 방법1: 특성이름으로 데이터셋 나누기
X=data[['gender_female', 'gender_male', 'region_Chungcheung', 'region_Honam',
        'region_Others', 'region_Sudo', 'region_Youngnam', 'edu', 'income',
        'age', 'score_gov', 'score_progress', 'score_intention']]
# 방법2: 특성 위치값으로 데이터셋 나누기
X=data[data.columns[1:14]]
# 방법3: loc 함수로 데이터셋 나누기 (단, 불러올 특성이 연달아 있어야 함)
X=data.loc[:, 'gender_female':'score_intention']


In [6]:
# 레이블 변수중 vote(투표여부) 데이터셋 나누기
y = data[['vote']]

## 5-2. train-test 데이터셋 나누기


In [8]:
# 훈련데이터와 테스트 데이터 셋 나누기 및 데이터 확인
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state =42)

In [10]:
# 데이터셋 행렬 구조 확인
print(X_train.shape)
print(X_test.shape)

# 골고루 잘 나눠졌는지
print(y_train.mean())
print(y_test.mean())

(158, 13)
(53, 13)
vote    0.708861
dtype: float64
vote    0.716981
dtype: float64


## 5-3. 모델 적용

### 가. 랜덤없는 교차검증: cross_val_score
- train데이터를 한번에 사용하지 않고 5개 그룹으로 나누어 이 중 한 그룹을 빼고 4개 그룹만 훈련
- 이렇게 5번 반복
- 총 158개이므로 1그룹은 약 30개 한번에 (30x4그룹)120개로 훈련

In [11]:
# LogisticRegression 알고리즘 사용
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [12]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_train, y_train, cv =5)
print("5개 테스트 셋 정확도:",scores)
print("정확도 평균:", scores.mean())

5개 테스트 셋 정확도: [0.71875    0.6875     0.8125     0.58064516 0.80645161]
정확도 평균: 0.7211693548387096


### 나. 랜덤 있는 교차검증: K-Fold

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold =KFold(n_splits=5, shuffle=True, random_state=42)
score = cross_val_score(model, X_train, y_train, cv =kfold)
print("5개 폴드 정확도:", score)

5개 폴드 정확도: [0.71875    0.6875     0.625      0.70967742 0.77419355]


### 다. 임의분할 교차검증 (ShuffleSplit)
- 훈련데이터와 테스트 데이터 구성할 때 다른 교차검증에 사용되었던 데이터도 랜덤으로 선택되게 하는 방법

In [14]:
from sklearn.model_selection import ShuffleSplit
shuffle_split = ShuffleSplit(test_size=0.5, train_size=0.5, random_state=42)
score = cross_val_score(model, X_train, y_train, cv = shuffle_split)
print("임의분할 교차검증 정확도:",score)

임의분할 교차검증 정확도: [0.73417722 0.69620253 0.70886076 0.73417722 0.65822785 0.67088608
 0.72151899 0.65822785 0.69620253 0.70886076]


## 5-4. train-validity-test 분할과 교차검증

In [15]:
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X,y,random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_val,y_train_val, random_state= 42)

In [16]:
model.fit(X_train,y_train)
scores = cross_val_score(model, X_train, y_train, cv =5)
print("교차검증 정확도:", scores)
print("정확도 평균:", scores.mean())

교차검증 정확도: [0.70833333 0.54166667 0.70833333 0.65217391 0.69565217]
정확도 평균: 0.661231884057971


In [17]:
# 검증데이터 정확도
model.score(X_valid, y_valid)

0.725

In [18]:
model.score(X_test,y_test)

0.7735849056603774